#### Quebec
Retrieved from: https://www.mfa.gouv.qc.ca/fr/services-de-garde/parents/localisateur/Pages/liste.aspx

In [1]:
import numpy as np
import pandas as pd

import re
import requests
import time
from bs4 import BeautifulSoup

*Scraper info*

In [2]:
headers = {
    'User-Agent': 'kt 1.0',
    'From': 'khobbs3@uwo.ca'
}

In [3]:
r = requests.get('https://www.mfa.gouv.qc.ca/fr/services-de-garde/parents/localisateur/Pages/liste.aspx', headers = headers)
s = BeautifulSoup(r.text)

In [4]:
domain = 'https://www.mfa.gouv.qc.ca'
cc = []
links = []


childcare = s.find_all('a', {'title':re.compile("Répertoire des services de garde par région administrative —?")})
for centre in childcare:
    cc.append(centre.text)
    links.append(domain + centre['href'])

In [5]:
links

['https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire1-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire2-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire3-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire4-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire5-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire6-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire7-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire8-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire9-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire10-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire11-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire12-sdg.xlsx',
 'https://www.mfa.gouv.qc.ca/fr/publication/Documents/repertoire13-sdg.xl

Download and concat each link
* CSV Tab 1: CPE and GARD
* CSV Tab 2: BC-MF

In [6]:

qc = pd.read_excel(links[0], header = 3, skipfooter = 4)
qc2 = pd.read_excel(links[0], sheet_name = "BC-MF", header = 3, skipfooter = 8)
master = pd.concat([qc, qc2], ignore_index=True)

master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Nom de l'installation         56 non-null     object 
 1   Région administrative         56 non-null     object 
 2   Adresse                       56 non-null     object 
 3   Municipalité                  56 non-null     object 
 4   Code postal                   56 non-null     object 
 5   Téléphone                     56 non-null     object 
 6   Télécopieur                   49 non-null     object 
 7   Adresse courriel              56 non-null     object 
 8   Type de service de garde      56 non-null     object 
 9   Places totales au permis      48 non-null     float64
 10  Places 17 mois ou moins       48 non-null     float64
 11  Place à contribution réduite  51 non-null     object 
 12  Territoire couvert            8 non-null      object 
 13  Places 

In [7]:
for link,c in zip(links[1::],cc[1::]):
    addme = pd.read_excel(link, header = 3, skipfooter = 4)
    addme2 = pd.read_excel(links[0], sheet_name = "BC-MF", header = 3, skipfooter = 8)
    full = pd.concat([addme, addme2], ignore_index=True)
    master = pd.concat([master, addme], ignore_index=True)

In [8]:
cols = ['Nom de l\'installation', 'Région administrative', 'Adresse',
       'Municipalité', 'Code postal', 'Téléphone', 'Télécopieur',
       'Adresse courriel', 'Type de service de garde',
       'Places totales au permis', 'Places 17 mois ou moins',
       'Place à contribution réduite', 'Territoire couvert',
       'Places totales à l\'agrément', 'Nombre de RSG reconnues']
for c in cols:
    try:
        master[c] = master[c].str.encode('utf-8-sig', 'ignore').str.decode('utf-8-sig')
    except AttributeError:
        print(c)

Places totales au permis
Places 17 mois ou moins
Places totales à l'agrément
Nombre de RSG reconnues


In [17]:
master.to_csv('data/childcare/QC-CPE-GARD-MF.csv', encoding = 'utf-8-sig')